In [12]:
import nibabel as nb

data_dir = 'MRIsamples/'
data_dir_save = data_dir + 'hdr_to_nii/'
data_ext = ['.img', '.hdr', '.nii']
filenames = ['4_8', '8_4', '100_23', '205_3']

for filename in filenames:
    try:
        #Se carga la imagen MRI simple en formato ANALYZE
        path = data_dir + filename
        img = nb.load(path + data_ext[0])
        hdr = nb.load(path + data_ext[1])

        #Se muestra la información del encabezado
        print(filename)
        print(hdr.header)
        print('*********************************************************************')
        
        #Se convierte el formato de ANALYZE a NIFTI
        nii = nb.nifti1.Nifti1Image(img.dataobj, hdr.affine, hdr.header)

        #Se guarda la imagen MRI simple en formao NIFTI
        path = path.replace(data_dir, data_dir_save) + data_ext[2]
        nb.save(nii, path)
    except FileNotFoundError:
        print(f"No se pudo cargar el archivo: '{filename}'")
        print('*********************************************************************')
    except AttributeError:
        print(f"El archivo: '{filename}', se encuentra vacio")
        print('*********************************************************************')

4_8
<class 'nibabel.spm2analyze.Spm2AnalyzeHeader'> object, endian='<'
sizeof_hdr     : 348
data_type      : b'dsr      '
db_name        : b'..\\work_t1\\ibsr\\o'
extents        : 0
session_error  : 0
regular        : b'r'
hkey_un0       : b'0'
dim            : [  4 256  61 256   1   0   0   0]
vox_units      : b'mm'
cal_units      : b''
unused1        : 0
datatype       : int16
bitpix         : 16
dim_un0        : 0
pixdim         : [0.  1.  3.1 1.  0.  0.  0.  0. ]
vox_offset     : 0.0
scl_slope      : nan
scl_inter      : 0.0
funused3       : 0.0
cal_max        : 0.0
cal_min        : 0.0
compressed     : 0
verified       : 0
glmax          : 32767
glmin          : 0
descrip        : b'spm compatible'
aux_file       : b'none                   '
orient         : b''
origin         : [91 34 91  0  0]
generated      : b''
scannum        : b''
patient_id     : b''
exp_date       : b''
exp_time       : b''
hist_un0       : b''
views          : 0
vols_added     : 0
start_field    : 0
field